In [107]:
import matplotlib.pyplot as plt

In [108]:
class Cell(object):
    """
    A representation of a game cell.
    """
    
    def __init__(self, position, cell_type, constraint):
        """
        Creates a game cell.
        
        :parameter position: the position of the cell.
        :type position: 2-dimensional tuple.
        :parameter cell_type: {'free', 'wall', 'constrained_wall'}.
        :type cell_type: string.
        :parameter constraint: number of lights that must be adjacent to the current cell.
        :type constraint: int.
        """
        self.position = position
        self.cell_type = cell_type
        self.constraint = constraint
        

In [109]:
class Board(object):
    """
    A representation of the game board.
    """
    
    def __init__(self, width, height, wall_positions):
        """
        Creates the game board.
        
        :parameter width: number of columns of the board.
        :type width: int.
        :parameter height: number of rows of the board.
        :type height: int.
        :parameter wall_positions: a list of wall positions.
        :type wall_positions: list of 3-dimensional tuple: (row, column, constraint).
        """
        self.board_width = width
        self.board_height = height
        self.board = []
        for i in range(self.board_height):
            self.board.append([])
            for j in range(self.board_width):
                self.board[i].append([])

        for i in range(width):
            for j in range(height):
                current_pos = (i, j)
                cell_type = 'free'
                constraint = None
                
                # Verify if current cell is a wall
                for wall_pos in wall_positions:
                    if i == wall_pos[0] and j == wall_pos[1]:
                        if wall_pos[2] is None:
                            cell_type = 'wall'
                        else:
                            cell_type = 'constrained_wall'
                            constraint = wall_pos[2]

                self.board[i][j] = Cell(current_pos, cell_type, constraint)
        
        self.light_positions = []
    
    def print_ascii_board(self):
        """
        Prints board using ASCII characters.
        F character is used for free cells.
        W character is used for unconstrained walls.
        Numbers are used for constrained walls. They indicate the number of adjacent lights required.
        """
        for i in range(self.board_width):
            for j in range(self.board_height):
                if self.board[i][j].cell_type == 'free':
                    print('F ', end='')
                elif self.board[i][j].cell_type == 'constrained_wall':
                    print(f'{self.board[i][j].constraint} ', end='')
                elif self.board[i][j].cell_type == 'wall':
                    print('W ', end='')
            print('\n')

In [110]:
width = 7
height = 7
wall_positions = [(0, 2, 1), 
                  (1, 3, 0), 
                  (1, 4, None), 
                  (2, 1, 0), 
                  (2, 6, 2), 
                  (3, 1, 0), 
                  (3, 3, None), 
                  (3, 5, None),
                  (4, 0, None),
                  (4, 5, 1),
                  (5, 2, 3),
                  (5, 3, 1),
                  (6, 4, None)]

board = Board(width, height, wall_positions)

In [106]:
board.print_ascii_board()

F F 1 F F F F 

F F F 0 W F F 

F 0 F F F F 2 

F 0 F W F W F 

W F F F F 1 F 

F F 3 1 F F F 

F F F F W F F 

